# Análise comparativa

## 1. Preparação de dados

### 1.1 Configurações iniciais

Importação de bibliotecas

In [47]:
from pathlib import Path
import pandas as pd
import numpy as np
import joblib
from IPython.display import Markdown
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.svm import SVC
from sklearn.model_selection import cross_validate, ShuffleSplit, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.impute import KNNImputer
from sklearn.preprocessing import RobustScaler
from sklearn.naive_bayes import BernoulliNB

Lendo o conjunto de dados

In [48]:
data_path = Path('../data/raw/hour.csv')
df = pd.read_csv(data_path)

Lendo dicionário de dados

In [49]:
dict_path = Path('../data/external/dicionario.csv')
df_dict = pd.read_csv(dict_path, sep=';')

### 1.2 Tratamento de dados

Para cada tipo de variável, será criado um pipeline para pré-processamento com os seguintes itens:
- Tratamento de dados discrepantes
- Tratamento de dados faltantes
- Codificação de variáveis
- Seleção de variáveis
- Normalização


Variável alvo

In [50]:
target_column = 'cnt'

Variáveis discretas

In [51]:
discrete_columns = (
    df_dict
    .query('Subtipo == "Discreta" and Variável != @target_column')
    ['Variável']
    .to_list()
)

discrete_columns

['instant', 'casual', 'registered']

Pipeline

In [52]:
discrete_preprocessor = Pipeline([
    # Tratamento de dados discrepantes
    ('missing', SimpleImputer(strategy='mean')), # Tratamento de dados faltantes
    # Seleção de variáveis
    ('normalization', MinMaxScaler()) # Normalização
])

Variáveis contínuas

In [53]:
continuous_columns = (
    df_dict
    .query('Subtipo == "Contínua"')
    ['Variável']
    .to_list()
)

continuous_columns

['temp', 'atemp', 'hum', 'windspeed']

Pipeline

In [54]:
continuous_preprocessor = Pipeline(steps=[
    # Tratamento de dados discrepantes
    ('missing', KNNImputer(n_neighbors=5)), # Tratamento de dados faltantes
    # Seleção de variáveis
    ('normalization', RobustScaler()) # Normalização
])

Variáveis nominais

In [55]:
nominal_columns = (
    df_dict
    .query('Subtipo == "Nominal"')
    ['Variável']
    .to_list()
)

nominal_columns

['holiday', 'workingday']

Pipeline

In [56]:
nominal_preprocessor = Pipeline([
    # Tratamento de dados discrepantes
    ('missing', SimpleImputer(strategy='most_frequent')), # Tratamento de dados faltantes
    ('encoder', OneHotEncoder(sparse=False)), # Codificação de variáveis
    # Seleção de variáveis
    ('normalization', MinMaxScaler()), # Normalização    
])

Variáveis ordinais

In [57]:
ordinal_columns = (
    df_dict
    .query('Subtipo == "Ordinal"')
    ['Variável']
    .to_list()
)

ordinal_columns

['dteday', 'season', 'yr', 'mnth', 'hr', 'weekday', 'weathersit']

Pipeline

In [58]:
ordinal_preprocessor = Pipeline([
    # Tratamento de dados discrepantes
    ('missing', SimpleImputer(strategy='most_frequent')), # Tratamento de dados faltantes
    ('encoder', OrdinalEncoder()), # Codificação de variáveis
    # Seleção de variáveis
    ('normalization', MinMaxScaler()), # Normalização    
])

Aplicando pré-processadores

In [59]:
preprocessing = ColumnTransformer([
    ("discrete", discrete_preprocessor, discrete_columns),
    # ("continuous", continuous_preprocessor, continuous_columns),
    # ("nominal", nominal_preprocessor, nominal_columns)
    ("ordinal", ordinal_preprocessor, ordinal_columns)
])

## 2. Escolha do modelo

### 2.1 Metodologia
Iremos análisar quatro modelos, que serão testados utilizando um método de validação cruzada por permutação, os modelos que serão testados serão: 

- Logistic Regression (LR)
- K-Nearest-Neighbors (KNN)
- Support Vector Machine (SVM)
- Naive Bayes (NB)

Além disso, cada um desses algoritmos será testado com diferentes parametros, para que possamos encontrar o melhor modelo e a melhor configuração possível para esse modelo.

Os modelos serão comparados através dos seguintes parâmetros
- `accuracy`: Proporção entre os dados que foram corretamente previstos (como positivos ou negativos) com o total de dados observados

- `precision` : Proporção entre dados corretamente previstos como positivos e o total de observações positivas. Ou seja, de todos os pacientes que identificamos com câncer, quantos realmente possuem?

- `recall`: Proporção entre dados corretamente previstos como positivos com o total de observações. Ou seja, entre todos os pacientes que possuem câncem pulmonar, quantos foram marcados? Em outras palavras, esse parâmetro ajuda a identificar a sensibilidade do nosso modelo

- `f1`: Média ponderada entre `precision` e `recall`, portanto levando em conta tanto falsos positivos quanto falsos negativos.

In [60]:
models = [
    (
        "LR", #Nome do modelo (abreviado)
        LogisticRegression(solver='saga', max_iter=1000), # Chamada do método do modelo
        {"penalty": ['none', 'l1', 'l2']} # Diferentes parametros que serão testados
    ),
    (
        "KNN",
        KNeighborsClassifier(metric='euclidean'),
        {"n_neighbors": np.arange(1, 31, 2), 'weights': ["uniform", "distance"]}
    ),
    (
        "SVM",
        SVC(max_iter=10000),
        {'C':[1, 10, 100, 1000],'gamma':[1, 0.1, 0.001, 0.0001], 'kernel':['linear','rbf']}
    ),
    (
        "NB",
        BernoulliNB(),
        {"alpha": [1e-3, 0.5, 1]}
    ),
]

### 2.2 Configuração do experimento
Iremos separar o conjunto de dados em conjunto de testes e de treino para realizar a validação cruzada

In [61]:
X = df.drop(columns=[target_column], axis=1)
y = (
    df[[target_column]]
    # .replace({"YES": 1, "NO": 0})
    .to_numpy()
    .ravel()
)

# Fazer a separação levando em conta a ordem dos dados
cv = ShuffleSplit(n_splits=30, train_size=0.8, random_state=42) #Separa em conjuntos de teste e treino

Agora iremos realizar o teste com os modelos definidos anteriormente, testando também os diferentes parâmetros para obter a melhor combinação possível. 

In [62]:
results = {}
for model_name, model, model_params in models:
    print(f'{model_name} run...')
    
    model_gs = GridSearchCV(model, model_params, scoring='accuracy')
    approach = Pipeline([
        ("preprocessing", preprocessing),
        ("model", model_gs)
    ])
    model_results = cross_validate(
        approach,
        X=X,
        y=y,
        scoring=['accuracy', 'f1', 'precision', 'recall'], #Critérios de avaliação
        cv=cv,
        n_jobs=-1,
        return_train_score=False
    )
    model_results['name'] = [model_name] * len(model_results['score_time'])
    if results:
        for key, value in model_results.items():
            results[key] = np.append(results[key], value)
    else:
        results = model_results

LR run...


/home/wesley/.cache/pypoetry/virtualenvs/src-JujeFAdE-py3.8/lib/python3.8/site-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
/home/wesley/.cache/pypoetry/virtualenvs/src-JujeFAdE-py3.8/lib/python3.8/site-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
/home/wesley/.cache/pypoetry/virtualenvs/src-JujeFAdE-py3.8/lib/python3.8/site-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
/home/wesley/.cache/pypoetry/virtualenvs/src-JujeFAdE-py3.8/lib/python3.8/site-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


KeyboardInterrupt: 

### 2.3 Resultados
Aqui iremos comparar os resultados e definir um modelo para ser utilizado no projeto

In [ ]:
df_results = pd.DataFrame(results)
df_results.groupby('name').agg([np.mean, np.std])

In [ ]:
#Cria uma tabela para melhor visualizar e comparar os modelos 

def highlight_max(s, props=''):
    values = [float(value.split()[0]) for value in s.values[1:]]
    result = [''] * len(s.values)
    if s.values[0].endswith('time'):
        result[np.argmin(values)+1] = props
    else:
        result[np.argmax(values)+1] = props
    return result

def get_winner(s):
    metric = s.values[0]
    values = [float(value.split()[0]) for value in s.values[1:]]
    models = results.columns[1:]
    
    if s.values[0].endswith('time'):
        return models[np.argmin(values)]
    else:
        return models[np.argmax(values)]

results = (
    pd
    .DataFrame(df_results)
    .groupby(['name'])
    .agg([lambda x: f"{np.mean(x):.3f} ± {np.std(x):.3f}"])#
    .transpose()
    .reset_index()
    .rename(columns={"level_0": "score"})
    .drop(columns="level_1")
    # .set_index('score')
)
time_scores = ['fit_time', 'score_time']
winner = results.query('score not in @time_scores').apply(get_winner, axis=1).value_counts().index[0]
results.columns.name = ''
results = (
    results
    .style
    .hide(axis='index')
    .apply(highlight_max, props='color:white;background-color:gray', axis=1)
)
display(results)
display(Markdown(f'### O melhor modelo é o : **{winner}**'))

### 2.5 Persistência do modelo
Dado que já obtivemos o melhor modelo agora podemos obter os melhores parâmetros desse modelo e salvar esse modelo em disco para utilizar na pŕóxima fase da análise

In [ ]:
#Obtem o modelo e os parametros ganhadores
model_name, model, model_params  = [foo for foo in models if foo[0] == winner][0] 

model_gs = GridSearchCV(model, model_params, scoring='accuracy')
approach = Pipeline([
    ("preprocessing", preprocessing),
    ("model", model_gs)
])
approach.fit(X, y) #Seleciona o approach

In [ ]:
joblib.dump(approach, '../models/model.joblib') # Salva o modelo em disco